import dependencies


In [2]:
#!pip install tensorflow opencv-python mediapipe sklearn matplotlib


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt # img show
import time
import tensorflow as tf
from tensorflow import keras
import mediapipe as mp

In [4]:
def mp_detection(image, model):
   # recolours frame from default BGR -> RGB
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # flips image so hands correct
        image = cv2.flip(image,1)
        # set flag
        image.flags.writeable = False
        # detect hands
        results = model.process(image)
        # sets flag back to true
        image.flags.writeable = True
        # recolours back to BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        return image,results

Set up mediapipe

In [5]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [6]:
def detection(image, model):
     # recolours frame from default BGR -> RGB
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # flips image so hands correct
        #image = cv2.flip(image,1)
        # set flag
        image.flags.writeable = False
        # detect hands
        results = model.process(image)
        # sets flag back to true
        image.flags.writeable = True
        # recolours back to BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        return image,results

In [7]:
def draw_landmarks(image, results):
    #Draws landmarks on image.
    # hand landmark setup left-right
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS,
    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=5),
    mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=8))

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS,
    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=5),
    mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=8))

In [8]:
def CaptureAndLandmark():
    capture = cv2.VideoCapture(0) # get video from webcam
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while capture.isOpened():
                    # read camera feed
                    ret, frame = capture.read()

                    # make detections
                    image, results = detection(frame, holistic) # passing in the feed alongside the model

                    # draw landmarks
                    draw_landmarks(image,results)
                    
                    # display camera feed
                    cv2.imshow('camera feed', image)

                    
                    # closes application
                    if cv2.waitKey(25) & 0xFF == ord('r'): # key for closing webcam frame
                            break

            capture.release()
            cv2.destroyAllWindows()


Basic feed and landmark display, no data collection

In [9]:
# Basic feed and landmark display, no data collection.
#CaptureAndLandmark()

keypoints for both hands

In [10]:
def keypoints(results):
    # extracting the results from the feed into an array which contains the relative positioning of the left and right hand
    # if hands are not present in the frame, populate an empty array with all zeros     
    left = np.array([[res.x,res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right = np.array([[res.x,res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # numpy concatenation sequences the arrays into one array
    return np.concatenate([left,right])

In [11]:
# returns the dimensions of the array, allows us to see if all data is correctly represented
# each hand array should return 21 points per axis, so, for each hand 21*3 = 63
# assuming all points are present, the shape of the array should be 126
# keypoints(results).shape

Data collection folders

In [12]:
# path for exported data
path = os.path.join('data_set')
# actions to be detected
actions=np.array(['1 finger','2 fingers','3 fingers','4 fingers','5 fingers'])
# number of videos
no_sequences=30
# length of frames
sequence_length=30

In [13]:
for act in actions:
    for sequence in range(no_sequences):
        # create a folder for each action, if the folder already exists then pass
        try:
            os.makedirs(os.path.join(path, act, str(sequence)))
        except:
            pass

In [14]:
capture = cv2.VideoCapture(0) # get video from webcam
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for act in actions:
                for sequence in range(no_sequences):
                        for frame_num in range(sequence_length):
                                # read camera feed
                                ret, frame = capture.read()

                                # make detections
                                image, results = detection(frame, holistic) # passing in the feed alongside the model

                                # draw landmarks
                                draw_landmarks(image,results)
                                
                                if frame_num == 0:
                                        cv2.putText(image, 'Start', (120,200),
                                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1,cv2.LINE_AA)

                                        cv2.putText(image, 'collecting for {} video number {}'.format(act, sequence),
                                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                                        # display camera feed
                                        cv2.imshow('camera feed', image)

                                        cv2.waitKey(500)
                                else:
                                        cv2.putText(image, 'collecting for {} video number {}'.format(act, sequence),
                                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                                        # display camera feed
                                        cv2.imshow('camera feed', image)

                                kp = keypoints(results)
                                npy_path = os.path.join(path, act, str(sequence), str(frame_num))
                                np.save(npy_path,keypoints)
                                
                               
                                # closes application
                                if cv2.waitKey(25) & 0xFF == ord('r'): # key for closing webcam frame
                                        break

        capture.release()
        cv2.destroyAllWindows()

KeyboardInterrupt: 

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
#Set up a map for the recorded actions
label_map = {label:num for num, label in enumerate(actions)}

In [17]:
import pickle
def load():
    try:
        with open(path, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return 0  # start with 0 if no storage present

In [18]:
def save(amount):
    with open(path, "wb") as file:
        pickle.dump(amount, file)

In [19]:
sequences, labels = [], []
# loop through all of the actions, for each action, loop through all of the frames.
for action in actions:
    for sequence in range(no_sequences):
        # creating a blank array to contain all the data in that particular sequence
        window = []
        for frame_num in range(sequence_length):
            # load the data from the npy files stored in the data collection folders
            res = np.load(os.path.join(path, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=true)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


PermissionError: [Errno 13] Permission denied: 'data set'